<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 3 / Assignment 1*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model 을 사용해보기
- <a href="#p2">Part 2:</a> Custom CNN Model을 제작해보기
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


케라스를 이용한 바이너리 이미지 분류 모델에 3가지 CNN 모델을 적용한다.  <br>
[데이터 다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/mountainForest.zip) <br>
산의 이미지(./data/mountin/*)와 숲의 이미지(./data/forest/*)를 분류하십시오. 산을 포스티브 클래스(1)로, 숲 이미지를 네거티브(0)로 처리한다.

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/codestates/ds-cs-section4-sprint3/blob/main/N431/data/mountain/art1131.jpg?raw=1)|![](https://github.com/codestates/ds-cs-section4-sprint3/blob/main/N431/data/forest/cdmc317.jpg?raw=1)|

표본이 작다는 점을 감안하면 문제는 현실적으로 어렵다. 즉, 클래스당 약 350개의 관측치가 있다. 이 샘플 크기는 직장에서 이미지 분류 문제/솔루션을 프로토타이핑할 때 기대할 수 있는 것일 수 있다. 이럴 때에는 여러가지 모델을 적용해보는 것이 중요하기 때문에 이번 sprint에서는 가능한 여러 모델을 평가하는 데 익숙해지는 것이 중요합니다.

# Part 1

## Pre - Trained Model
<a id="p1"></a>

Keras에서 제공하는 pre-trained 모델인 ResNet50을 불러오고, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - 이 모델은 50 개의 layer를 가진  CNN기반의 모델입니다. 총 [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt)를 분류하는 모델로 아래와 같이 사용할 수 있습니다. 우리가 풀어야 할 과제는 2가지 이니, 마지막 출력 단을 변경해서 사용하면 됩니다.

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

`ResNet50`을 불러올 때, `include_top`을 False로 하면, 기존 1000개의 분류 문제를 풀 수 있는 ResNet 모델에서 Fully Connected layer를 제거해주는 역할을 합니다. 

```python
for layer in resnet.layers:
    layer.trainable = False
```
이 부분은 ResNet50 레이어들의 파라미터 학습모드를 끄는(off) 것입니다. 이렇게 설정된 매개 변수는 설령 지금 순전파/역전파 이후 오류가 전파 되더라도 파라미터가 학습(업데이트)되지 않는 것입니다. 

Keras 기능 API를 사용하여 모델에 추가로 `Fully-conneted layer`(Dense모델)을 추가해야합니다. 우리는 최상위 레이어,  `Fully-conneted layer`를 제거합니다. 출력층의 숫자(=분류할 클래스의 개수)가 맞지 않기 때문이죠. 네트워크의 특징을 추출하는 부분 만 유지하는 것입니다. `GlobalAveragePooling2D` 레이어는 마지막 컨벌루션 레이어 출력 (2 차원) 각각의 평균을 취함으로써 정말 멋진 평탄화 기능으로 작동합니다.

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) # 출력층 설계
model = Model(res.input, predictions)
```

이 과제는 산의 이미지 (`. / data / mountain / *`)와 숲의 이미지 (`. / data / forest / *`)를 분류하기 위해 위의 전이 학습(transfer learning)을 적용하는 것입니다. 산을 postive 클래스 (1)로, 숲 이미지를 음수 (0)로 취급합니다.

Steps to complete assignment: 
1. 이미지 데이터를 numpy 배열 (`X`)로 불러옵니다.
2. 레이블에 대한 `y`를 만듭니다.
3. resnet의 사전 훈련 된(pre-trained) 레이어로 모델 훈련
4. 모델의 정확성보고

In [1]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

base_dir = '/content/drive/My Drive/N431'

train_dir = os.path.join(base_dir, 'train')
train_forest_dir = os.path.join(train_dir, 'forest')
train_mountain_dir = os.path.join(train_dir, 'mountain')

validation_dir = os.path.join(base_dir, 'validation')
validation_forest_dir = os.path.join(validation_dir, 'forest')
validation_mountain_dir = os.path.join(validation_dir, 'mountain')

In [4]:
train_forest_fnames = os.listdir(train_forest_dir)
train_forest_fnames.sort()
print(train_forest_fnames[:10])

train_mountain_fnames = os.listdir(train_mountain_dir)
train_mountain_fnames.sort()
print(train_mountain_fnames[:10])

['.DS_Store', '.ipynb_checkpoints', 'art114.jpg', 'for102.jpg', 'for105.jpg', 'for106.jpg', 'for110.jpg', 'for112.jpg', 'for114.jpg', 'for116.jpg']
['.DS_Store', '.ipynb_checkpoints', 'art1131.jpg', 'art1132.jpg', 'gre242.jpg', 'land10.jpg', 'land11.jpg', 'land13.jpg', 'land130.jpg', 'land131.jpg']


In [ ]:
print('total training forest images:', len(os.listdir(train_forest_dir)))
print('total training mountain images:', len(os.listdir(train_mountain_dir)))
print('total validation forest images:', len(os.listdir(validation_forest_dir)))
print('total validation mountain images:', len(os.listdir(validation_mountain_dir)))

In [8]:
from tensorflow.keras import layers

In [9]:
img_input = layers.Input(shape=(224*224*3))

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Found 533 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [11]:
for layer in resnet.layers:
    layer.trainable = False

In [12]:
x = resnet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [13]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [14]:
history = model.fit(
      train_generator,
      steps_per_epoch=27,  
      epochs=15,
      validation_data=validation_generator,
      validation_steps=10,
      verbose=1)

Epoch 1/15
27/27 [==============================] - 223s 7s/step - loss: 1.0484 - accuracy: 0.4713 - val_loss: 0.5902 - val_accuracy: 0.6667
Epoch 2/15
27/27 [==============================] - 3s 116ms/step - loss: 0.7410 - accuracy: 0.5315 - val_loss: 0.5521 - val_accuracy: 0.7333
Epoch 3/15
27/27 [==============================] - 3s 116ms/step - loss: 0.5540 - accuracy: 0.7387 - val_loss: 0.5056 - val_accuracy: 0.7795
Epoch 4/15
27/27 [==============================] - 3s 115ms/step - loss: 0.5163 - accuracy: 0.7720 - val_loss: 0.5259 - val_accuracy: 0.8154
Epoch 5/15
27/27 [==============================] - 3s 115ms/step - loss: 0.4618 - accuracy: 0.8660 - val_loss: 0.4310 - val_accuracy: 0.7949
Epoch 6/15
27/27 [==============================] - 3s 114ms/step - loss: 0.4219 - accuracy: 0.8540 - val_loss: 0.5192 - val_accuracy: 0.7795
Epoch 7/15
27/27 [==============================] - 3s 117ms/step - loss: 0.3792 - accuracy: 0.8714 - val_loss: 0.4927 - val_accuracy: 0.8000
Epoch 8

In [14]:
model.evaluate(X_validation, y_validation)

6/6 [==============================] - 2s 225ms/step - loss: 26.0850 - accuracy: 0.0000e+00


[26.084976196289062, 0.0]

# Part 2

## Custom CNN Model


이 단계에서는 Keras를 사용하여 자신 만의 CNN을 작성하고 훈련합니다. 네트워크 시작 부분에 적어도 하나의 Conv 레이어와 pooling 레이어가있는 아키텍처를 사용할 수 있습니다. 원하는 경우 더 추가 해볼 수 있습니다.

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [12]:
model_scratch = Sequential()
model_scratch.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model_scratch.add(MaxPooling2D(2,2))
model_scratch.add(Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model_scratch.add(MaxPooling2D(2,2))
model_scratch.add(Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model_scratch.add(Flatten())
model_scratch.add(Dense(128, activation='relu'))
model_scratch.add(Dense(1, activation='sigmoid'))

In [13]:
# Compile Model
model_scratch.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
# Fit Model
model_scratch.fit(X_train, y_train, batch_size=32, epochs=15, validation_data=(X_validation, y_validation))